<a href="https://colab.research.google.com/github/ChangLuyue/git-blog-demo/blob/master/WEEK8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
os.chdir('/content/drive/MyDrive/project')
adapt_data = pd.read_csv('adapt_data.csv')
adapt_data_sample_size = pd.read_csv('adapt_data_sample_size.csv')

Mounted at /content/drive


In [38]:
def adapt(T,S,acc):
  S_last = S
  S_final = S
  S_step_3 = S
  S_loop = S
  S_while = S
  time_budget_schedule = T
  # time_budget_schedule_loop = T
  baseline_list = []
  # Get needed data from csv and name it indexed_new_df
  group_sample_size = adapt_data_sample_size.loc[:,['sample_size','nmf']].groupby(['sample_size']).count()
  # Combine the counted number with the previous csv and rename the column
  group_sample_size_count = adapt_data_sample_size.merge(group_sample_size,on='sample_size',left_index=True)
  group_sample_size_count.rename(columns={'nmf_x':'nmf','nmf_y':'changed_value'},inplace=True)
  # create a new dataframe contains nmf\sample_size\changed_value column
  new_df = group_sample_size_count.loc[:,['nmf','sample_size','changed_value']]
  # reset index of the dataframe and delete original index
  indexed_new_df = new_df.reset_index(drop=True)
  indexed_new_df

# --------------------------------------------------------------------------------------------
  # Make a prediction dictionary according to the first value of S
  update_s = indexed_new_df[indexed_new_df['sample_size']==S]['changed_value'].iloc[0]
  predict_base = indexed_new_df['nmf'][S-update_s]
  # print('predict_base:',predict_base)
  
  predict_first = predict_base
  key = []
  value = []
  value.append(predict_base)

  while S > 0:
    key.append(S)
    predict_value = predict_base*(S-update_s)/S
    value.append(predict_value)
    S -= update_s
    predict_base = predict_value
  value.pop()
  predicted_dic = dict(zip(key,value))
  print('predicted_dic:',predicted_dic)
# -------------------------------------------------------------------------------------

  # step 3.1
  # Compute schedule 
  baseline = predicted_dic[list(predicted_dic.keys())[-1]]
  baseline_list.append(baseline)
  # The time_budget here is defined by T minus first subsample of S
  time_budget = T - predict_first
  # In the loop, the first S and the other S is different
  p_key = []
  for k in predicted_dic.keys():
    p_key.append(k)
  p_key.pop(0)

  # fill schedule
  actual_dic = {}
  i = 1
  time_budget2 = time_budget
  while time_budget2 > baseline:
    actual_dic[S_step_3] = i
    if i >= 2:
      time_budget2 -= predicted_dic[S_step_3]
    # if time_budget>0,the number of subsamples will add 1,else minus 1
      for j in p_key:
        time_budget2 -= predicted_dic[j]
        if time_budget2 >= 0:
          actual_dic[j] = i
        else:
          actual_dic[j] = i-1
      i += 1
    else:
      for j in p_key:
        time_budget2 -= predicted_dic[j]
        if time_budget2 >= 0:
          actual_dic[j] = i
        else:
          actual_dic[j] = i-1
      i += 1


  for k in actual_dic.keys():
    # if number of sub-samples is bigger than the total number of sub-samples
    # then we set it as the total number of sub-sample. 
    if actual_dic[k] > indexed_new_df[indexed_new_df['sample_size']==k]['changed_value'].iloc[0]:
      actual_dic[k] = indexed_new_df[indexed_new_df['sample_size']==k]['changed_value'].iloc[0]
  print('schedule:',actual_dic)

  # compute time_budget by real values from csv according to schedule
  for key,value in actual_dic.items():
    time_budget_schedule -= np.sum(indexed_new_df[indexed_new_df['sample_size']==key][0:value]['nmf'])
  print('The first time budget:',time_budget_schedule)

  # Step 3.2 : Play out computed schedule 
  # make an accuracy dictionary
  accuracy = 0
  acc_dic = {}
  actual_dictionary = {}
  for i,j in actual_dic.items():
    current_time_list = []
    for p in range(0,j):
      # while accuracy < acc :
      current_time_list.append(indexed_new_df[indexed_new_df['sample_size']==i]['nmf'].iloc[p])
      actual_dictionary[i] = current_time_list
    # new change ------------------------------------------------
      acc_dic[i] = np.std(actual_dictionary[i])/np.mean(actual_dictionary[i])
      if p >= 2 and acc_dic[i] <= acc:
       actual_dic[i] = p + 1;
       break;

  print('actual_dictionary:',actual_dictionary)
  print('acc_dic:',acc_dic)

  # update prediction dictionary
  key = []
  value = []
  # get the maximum of the first list in the actual_dictionary
  predict_new = max(actual_dictionary[S_final])
  value.append(predict_new)
  S_last = S_final
  while S_last > 0:
    key.append(S_last)
    predict_value = predict_new*(S_last-update_s)/S_last
    value.append(predict_value)
    S_last -= update_s
    predict_new = predict_value
  value.pop()
  predicted_dic = dict(zip(key,value))
  print('updated_predicted_dic:',predicted_dic)
  baseline = predicted_dic[list(predicted_dic.keys())[-1]]
  time_budget_schedule_list = []
  while time_budget_schedule > baseline and len(time_budget_schedule_list)==len(set(time_budget_schedule_list)):
    S_loop = S_while
    while S_loop > 0:
      # new change-------------------------------------------
      for i,j in acc_dic.items():
        if acc_dic[i] > acc:
          if time_budget_schedule >= predicted_dic[S_loop]:
            actual_dic[S_loop] += 1
            time_budget_schedule -= predicted_dic[S_loop]
            S_loop -= indexed_new_df[indexed_new_df['sample_size']==S_loop]['changed_value'].iloc[0]
          else:
            S_loop -= indexed_new_df[indexed_new_df['sample_size']==S_loop]['changed_value'].iloc[0]
        else:
          actual_dic[S_loop] += 0
          time_budget_schedule -= 0
          S_loop -= indexed_new_df[indexed_new_df['sample_size']==S_loop]['changed_value'].iloc[0]

    for k in actual_dic.keys():
      # if number of sub-samples is bigger than the total number of sub-samples
      # then we set it as the total number of sub-sample. 
      if actual_dic[k] > indexed_new_df[indexed_new_df['sample_size']==k]['changed_value'].iloc[0]:
        actual_dic[k] = indexed_new_df[indexed_new_df['sample_size']==k]['changed_value'].iloc[0]
    print('updated schedule',actual_dic)


    schedule_keys = []
    for schedule_key in actual_dic.keys():
      schedule_keys.append(schedule_key)

    if len(set(actual_dic.values())) == 1:
      different_S = S_while
    else:
      prep_i = []
      for i in range(0,len(schedule_keys)):
        if actual_dic[schedule_keys[i]] != actual_dic[schedule_keys[i-1]]:
          prep_i.append(i)
        else:
          different_S = S_while
      different_S = schedule_keys[prep_i[1]]

    actual_dictionary = {}
    for i,j in actual_dic.items():
      current_time_list = []
      for p in range(0,j):
        current_time_list.append(indexed_new_df[indexed_new_df['sample_size']==i]['nmf'].iloc[p])
  # accuracy or average /add dictionary 加一个字典accuracy threshold
        actual_dictionary[i] = current_time_list
        acc_dic[i] = np.std(actual_dictionary[i])/np.mean(actual_dictionary[i])
        if p >= 2 and acc_dic[i] <= acc:
          actual_dic[i] = p + 1;
          break;

    print('actual_dictionary:',actual_dictionary)
    print('upadated acc_dic',acc_dic)

    time_budget_schedule_loop = T
    for key,value in actual_dic.items():
      time_budget_schedule_loop -= np.sum(indexed_new_df[indexed_new_df['sample_size']==key][0:value]['nmf'])

    # update prediction dictionary
    key = []
    value = []
    # get the maximum of the first list in the actual_dictionary
    predict_new = max(actual_dictionary[S_final])
    value.append(predict_new)
    S_last = S_final
    while S_last > 0:
      key.append(S_last)
      predict_value = predict_new*(S_last-update_s)/S_last
      value.append(predict_value)
      S_last -= update_s
      predict_new = predict_value
    value.pop()
    predicted_dic = dict(zip(key,value))
    print('updated_predicted_dic:',predicted_dic)

    # update baseline
    baseline = predicted_dic[list(predicted_dic.keys())[-1]]
    print('the value of baseline after update:',baseline)
    # update time budget
    time_budget_schedule = time_budget_schedule_loop
    time_budget_schedule_list.append(time_budget_schedule)
    print('remaining time after update:',time_budget_schedule)
  print('The final shedule',actual_dic)

In [39]:
adapt(100,20,0.01)

predicted_dic: {20: 1.7026009999999998, 10: 0.8513005}
schedule: {20: 10, 10: 10}
The first time budget: 78.477886
actual_dictionary: {20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218, 1.669371, 1.65777, 1.778943, 1.744434, 1.614781, 1.687775], 10: [0.459082, 0.463218, 0.456826]}
acc_dic: {20: 0.026730737328819577, 10: 0.00575773043421017}
updated_predicted_dic: {20: 1.778943, 10: 0.8894715}
updated schedule {20: 10, 10: 3}
actual_dictionary: {20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218, 1.669371, 1.65777, 1.778943, 1.744434, 1.614781, 1.687775], 10: [0.459082, 0.463218, 0.456826]}
upadated acc_dic {20: 0.026730737328819577, 10: 0.00575773043421017}
updated_predicted_dic: {20: 1.778943, 10: 0.8894715}
the value of baseline after update: 0.8894715
remaining time after update: 81.747489
updated schedule {20: 10, 10: 3}
actual_dictionary: {20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218, 1.669371, 1.65777, 1.778943, 1.744434, 1.614781, 1.

In [ ]:
adapt(400,70,0.04)

predicted_dic: {70: 23.407132999999998, 60: 20.063256857142857, 50: 16.719380714285716, 40: 13.375504571428573, 30: 10.03162842857143, 20: 6.687752285714287, 10: 3.3438761428571433}
schedule: {70: 5, 60: 4, 50: 4, 40: 4, 30: 4, 20: 4, 10: 4}
The first time budget: 107.96573099999999
actual_dictionary: {70: [23.407132999999998, 25.065826, 23.334456, 26.303234999999997, 23.307528], 60: [17.055281, 17.063126999999998, 17.166774, 17.216573999999998], 50: [11.869168, 11.726266, 11.907739, 11.880541000000001], 40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.9523589999999995], 20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218], 10: [0.459082, 0.463218, 0.456826, 0.451071]}
acc_dic: {70: 0.049800861197316866, 60: 0.004005232438791348, 50: 0.005950925646461742, 40: 0.03077733432481601, 30: 0.01706078978910692, 20: 0.013373927617343125, 10: 0.009587527400300735}
updated_predicted_dic: {70: 2

In [ ]:
adapt(400,70,0.004)

predicted_dic: {70: 23.407132999999998, 60: 20.063256857142857, 50: 16.719380714285716, 40: 13.375504571428573, 30: 10.03162842857143, 20: 6.687752285714287, 10: 3.3438761428571433}
schedule: {70: 5, 60: 4, 50: 4, 40: 4, 30: 4, 20: 4, 10: 4}
The first time budget: 107.96573099999999
actual_dictionary: {70: [23.407132999999998, 25.065826, 23.334456, 26.303234999999997, 23.307528], 60: [17.055281, 17.063126999999998, 17.166774, 17.216573999999998], 50: [11.869168, 11.726266, 11.907739, 11.880541000000001], 40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.9523589999999995], 20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218], 10: [0.459082, 0.463218, 0.456826, 0.451071]}
acc_dic: {70: 0.049800861197316866, 60: 0.004005232438791348, 50: 0.005950925646461742, 40: 0.03077733432481601, 30: 0.01706078978910692, 20: 0.013373927617343125, 10: 0.009587527400300735}
updated_predicted_dic: {70: 2

In [ ]:
adapt(400,70,0.9)

predicted_dic: {70: 23.407132999999998, 60: 20.063256857142857, 50: 16.719380714285716, 40: 13.375504571428573, 30: 10.03162842857143, 20: 6.687752285714287, 10: 3.3438761428571433}
schedule: {70: 5, 60: 4, 50: 4, 40: 4, 30: 4, 20: 4, 10: 4}
The first time budget: 107.96573099999999
actual_dictionary: {70: [23.407132999999998, 25.065826, 23.334456, 26.303234999999997, 23.307528], 60: [17.055281, 17.063126999999998, 17.166774, 17.216573999999998], 50: [11.869168, 11.726266, 11.907739, 11.880541000000001], 40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.9523589999999995], 20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218], 10: [0.459082, 0.463218, 0.456826, 0.451071]}
acc_dic: {70: 0.049800861197316866, 60: 0.004005232438791348, 50: 0.005950925646461742, 40: 0.03077733432481601, 30: 0.01706078978910692, 20: 0.013373927617343125, 10: 0.009587527400300735}
updated_predicted_dic: {70: 2

In [ ]:
adapt(400,70,0.01)

predicted_dic: {70: 23.407132999999998, 60: 20.063256857142857, 50: 16.719380714285716, 40: 13.375504571428573, 30: 10.03162842857143, 20: 6.687752285714287, 10: 3.3438761428571433}
schedule: {70: 5, 60: 4, 50: 4, 40: 4, 30: 4, 20: 4, 10: 4}
The first time budget: 107.96573099999999
actual_dictionary: {70: [23.407132999999998, 25.065826, 23.334456, 26.303234999999997, 23.307528], 60: [17.055281, 17.063126999999998, 17.166774, 17.216573999999998], 50: [11.869168, 11.726266, 11.907739, 11.880541000000001], 40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.9523589999999995], 20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218], 10: [0.459082, 0.463218, 0.456826, 0.451071]}
acc_dic: {70: 0.049800861197316866, 60: 0.004005232438791348, 50: 0.005950925646461742, 40: 0.03077733432481601, 30: 0.01706078978910692, 20: 0.013373927617343125, 10: 0.009587527400300735}
updated_predicted_dic: {70: 2

In [ ]:
adapt(100,40,0.01)

predicted_dic: {40: 8.050094999999999, 30: 6.037571249999999, 20: 4.0250474999999994, 10: 2.0125237499999997}
schedule: {40: 5, 30: 5, 20: 5, 10: 4}
The first time budget: 31.852393
actual_dictionary: {40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001, 7.700977000000001], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.9523589999999995, 4.046638], 20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218, 1.669371], 10: [0.459082, 0.463218, 0.456826, 0.451071]}
acc_dic: {40: 0.027632256856809336, 30: 0.021429740512005375, 20: 0.012246205519577452, 10: 0.009587527400300735}
updated_predicted_dic: {40: 8.050094999999999, 30: 6.037571249999999, 20: 4.0250474999999994, 10: 2.0125237499999997}
updated schedule {40: 6, 30: 6, 20: 6, 10: 4}
actual_dictionary: {40: [8.050094999999999, 7.546710000000001, 7.538053, 7.454207000000001, 7.700977000000001, 7.4049830000000005], 30: [3.8059559999999997, 3.9692199999999995, 3.8635129999999998, 3.95235899999

In [31]:
adapt(100,20,0.1)

predicted_dic: {20: 1.7026009999999998, 10: 0.8513005}
schedule: {20: 10, 10: 10}
The first time budget: 78.477886
actual_dictionary: {20: [1.7026009999999998, 1.660134, 1.7023580000000003, 1.655218, 1.669371, 1.65777, 1.778943, 1.744434, 1.614781, 1.687775], 10: [0.459082, 0.463218, 0.456826, 0.451071, 0.468164, 0.5001899999999999, 0.435992, 0.460204, 0.479109, 0.47487299999999993]}
acc_dic: {20: 0.026730737328819577, 10: 0.03540036970940577}
updated_predicted_dic: {20: 1.778943, 10: 0.8894715}
updated schedule {20: 10, 10: 10}
actual_dictionary: {20: [1.7026009999999998], 10: [0.459082]}
upadated acc_dic {20: 0.0, 10: 0.0}
updated_predicted_dic: {20: 1.7026009999999998, 10: 0.8513005}
the value of baseline after update: 0.8513005
remaining time after update: 97.838317
updated schedule {20: 1, 10: 1}
actual_dictionary: {20: [1.7026009999999998], 10: [0.459082]}
upadated acc_dic {20: 0.0, 10: 0.0}
updated_predicted_dic: {20: 1.7026009999999998, 10: 0.8513005}
the value of baseline afte